In [33]:
'''
    Load RESNET model and split it
        1. layer by layer
        2. [TODO] vertically 
'''

'\n    Load RESNET model and split it\n        1. layer by layer\n        2. [TODO] vertically \n'

In [34]:
from source.core.engine import MoP
import source.core.run_partition as run_p
from os import environ
from source.utils.dataset import *
from source.utils.misc import *
from split_network import *

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from source.models import resnet

import torch.nn.functional as F

import numpy as np

from source.utils import io
from source.utils import testers
from source.core import engine
import json
import itertools

from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchsummary import summary

import time

In [35]:
# setup config
dataset='cifar10'
environ["config"] = f"config/{dataset}.yaml"

configs = run_p.main()

configs["device"] = "cpu"
#configs['load_model'] = "cifar10-resnet18-kernel-npv0-pr0.75-lcm0.001.pt"
configs['load_model'] = "cifar10-resnet18-kernel-npv2-pr0.75-lcm0.001.pt"
configs["num_partition"] = '4' #'resnet18-v2.yaml'

device :  
model :  resnet18
data_code :  cifar10
num_classes :  10
model_file :  test.pt
epochs :  0
batch_size :  128
optimizer :  sgd
lr_scheduler :  default
learning_rate :  0.01
seed :  1234
sparsity_type :  kernel
prune_ratio :  1
admm :  True
admm_epochs :  300
rho :  0.0001
multi_rho :  True
retrain_bs :  128
retrain_lr :  0.005
retrain_ep :  50
retrain_opt :  default
xentropy_weight :  1.0
warmup :  False
warmup_lr :  0.001
warmup_epochs :  10
mix_up :  True
alpha :  0.3
smooth :  False
smooth_eps :  0
save_last_model_only :  False
num_partition :  1
layer_type :  regular
bn_type :  masked
par_first_layer :  False
comm_outsize :  False
lambda_comm :  0
lambda_comp :  0
distill_model :  
distill_loss :  kl
distill_temp :  30
distill_alpha :  1


In [36]:
# load data and load or train model
model = get_model_from_code(configs).to(configs['device']) # grabs model architecture from ./source/models/escnet.py
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [37]:
# load weights into full model
state_dict = torch.load(io.get_model_path("{}".format(configs["load_model"])), map_location=configs['device'])
model = io.load_state_dict(model, 
                    state_dict['model_state_dict'] if 'model_state_dict' in state_dict 
                    else state_dict['state_dict'] if 'state_dict' in state_dict else state_dict,)


In [38]:
'''
    add partitions and communications to configs
'''

# gets random test input (with correct size)
input_var = engine.get_input_from_code(configs)
#print(input_var)

# Config partitions and prune_ratio
configs['num_partition'] = '4'#'./config/resnet18-v2.yaml'
configs = engine.partition_generator(configs, model)
            
# Compute output size of each layer
configs['partition'] = engine.featuremap_summary(model, configs['partition'], input_var)
        
# Setup communication costs
configs['comm_costs'] = engine.set_communication_cost(model, configs['partition'],)


# split model general parameters

# make copies of model per machine
num_machines = max(configs['partition']['bn_partition']) # TODO: double check this makes sense
model_machines = [model]*num_machines

layer_names_fx =  get_graph_node_names(model)[1]
total_layers_fx = len(layer_names_fx)

split_module_names = list(configs['partition'].keys())

print(layer_names_fx)


Inference time per data is 17.376900ms.
conv1.weight 1024
layer1.0.conv1.weight 1024
layer1.0.conv2.weight 1024
layer1.1.conv1.weight 1024
layer1.1.conv2.weight 1024
layer2.0.conv1.weight 256
layer2.0.conv2.weight 256
layer2.0.shortcut.0.weight 256
layer2.1.conv1.weight 256
layer2.1.conv2.weight 256
layer3.0.conv1.weight 64
layer3.0.conv2.weight 64
layer3.0.shortcut.0.weight 64
layer3.1.conv1.weight 64
layer3.1.conv2.weight 64
layer4.0.conv1.weight 16
layer4.0.conv2.weight 16
layer4.0.shortcut.0.weight 16
layer4.1.conv1.weight 16
layer4.1.conv2.weight 16
['x', 'conv1', 'bn1', 'relu', 'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu', 'layer1.0.conv2', 'layer1.0.bn2', 'layer1.0.add', 'layer1.0.relu_1', 'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu', 'layer1.1.conv2', 'layer1.1.bn2', 'layer1.1.add', 'layer1.1.relu_1', 'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.shortcut.0', 'layer2.0.shortcut.1', 'layer2.0.add', 'layer2.0.relu_1', 'la

In [43]:
''' Prep Inout for Mock Run version 2 (fx)'''

# TODO: reduce size of communicated tensors to only what is necessary 
# TODO: also check bias for nonzero
# TODO: come up with more general scheme to handle residual layers

# channel_id == INPUTS
# filter_id  == OUTPUTS

# try greater precision
#torch.set_default_tensor_type(torch.DoubleTensor)
#torch.set_default_dtype(torch.float64)

# setup input 
N_batch = 1
torch.manual_seed(0)
input_tensor = torch.rand(N_batch, 3, 32, 32, dtype=torch.float64, device=torch.device(configs['device'])) # 1k images, 3 channels, 32x32 image (cifar100) 
mdoel = model.type(torch.float64)


# broadcast input_tensor to different machines
# TODO: find a better datastructure for this
#input = np.empty((num_machines, num_machines), dtype=torch.Tensor)
input = [None]*num_machines
input = [input[:] for i in range(num_machines)]
for imach in range(num_machines):
    input[imach][imach] = input_tensor

residual_block_start, residual_connection_start, residual_block_end = get_residual_block_indexes(model)

# put models into eval mode and on device
model.eval()
model.to(configs['device'])

# set index to execute vertical splitting up to 
exec_to_layer = 69

# get true output at each layer
horz_output, size_LUT = get_output_at_each_layer(model, input_tensor)

BREAK_LOOP = 0 # break loop when output differs
PROBLEM_LAYER = 4

'''
    mock run through inference using split models 
'''

# timing
split_execution_start_time = time.time()
layer_completion_time_stamp = {}
layer_execution_duration = {}

# make inference 
with torch.no_grad():
    residual_input = {} # use this to keep track of inputs stored in machine memory for residule layers

    # iterate through layers 1 module at a time 
    for imodule in range(exec_to_layer+1):#range(num_total_modules): # 16 <=> layer_1 block 

        # DEBUGGING
        #if imodule == PROBLEM_LAYER:
        #    input = replace_all_tensors(input, 1)

        # initialize output for ilayer
        #output = np.empty((num_machines, num_machines), dtype=torch.Tensor) # square list indexed as: output[destination/RX machine][origin/TX machine]
        # TODO: find a better datastructure for this 

        output = [None]*num_machines
        output = [output[:] for i in range(num_machines)]

        # DEBUG
        full_input = combine_all_inputs(input, num_machines)

        print(f'Executing module {imodule}: {layer_names_fx[imodule]}')

        # iterate through each machine (done in parallel later)
        for imach in range(num_machines):
            print(f'\tExecuting on machine {imach}')

            # collect communication inputs if necessary 
            if not imodule == 0 and ('conv' in layer_names_fx[imodule-1] or 'linear' in layer_names_fx[imodule-1] or 'shortcut.1' in layer_names_fx[imodule]): # TODO: this is very hacky, needs to be generalized. The issue is ID'ing conv layers in shortcut blocks
                curr_input = combine_inputs(input, num_machines, imach)
            else:
                curr_input = input[imach][imach]

            # skip this machine+module if there is no input to compute 
            if not torch.is_tensor(curr_input):
                if 'bn' in layer_names_fx[imodule]:
                    print('\t\t-No input received but bn still needs to produce output.')
                    curr_input = torch.zeros(size_LUT[layer_names_fx[imodule]], dtype=torch.float64,  device=torch.device(configs['device']))
                else:
                    print('\t\t-No input sent to this machine. Skipping module')
                    continue
            
            # debug
            print(f'\t\t received input channels {get_nonzero_channels(curr_input)}')

            # non-comms operations 
            if 'relu' in layer_names_fx[imodule]:
                # just relu no comm necessary 
                print('\t\t-Applying ReLU')
                output[imach][imach] = F.relu(curr_input)
                continue

            elif 'add' in layer_names_fx[imodule]:
                # residual layer. No comm necessary 
                if str(imach) in residual_input:
                    print('\t\t-adding residual')
                    if 'block_out' in residual_input[str(imach)]:
                        curr_input = curr_input + residual_input[str(imach)]['block_out']
                    elif 'block_in' in residual_input[str(imach)]:
                        curr_input = curr_input + residual_input[str(imach)]['block_in']
                        print('\t\t-assuming shortcut had no layers')
                else:
                    print(f'\t\t-assuming this machine did not rx any input at the beginning of this block. No residual found')
                output[imach][imach] = curr_input

                # erase stored 
                residual_input[str(imach)] = {}
                continue
            
            elif 'avg_pool2d' in layer_names_fx[imodule]:
                print('\t\t-average pooling')
                output[imach][imach] = F.avg_pool2d(curr_input, 4)
                continue
            
            elif 'size' in layer_names_fx[imodule]:
                print('\t\t-skipping')
                output[imach][imach] = curr_input
                continue
            
            elif 'view' in layer_names_fx[imodule]:
                print('\t\t-reshaping (view)')
                output[imach][imach] = curr_input.view(curr_input.size(0), -1)
                continue
            elif 'x' == layer_names_fx[imodule]:
                # do nothing if model input
                print('\t\t-model input layer.. skipping')
                output[imach][imach] = curr_input
                continue
                        
            # swap out io for residual connection
            if imodule in residual_block_start:
                # save input for later 
                residual_input[str(imach)] = {}
                residual_input[str(imach)]['block_in'] = curr_input.detach().clone()
                print('\t\t-Saving input for later...')
            elif imodule in residual_connection_start:
                # swap tensors
                residual_input[str(imach)]['block_out'] = curr_input
                curr_input = residual_input[str(imach)]['block_in'] 
                print('\t\t-Saving current input. Swapping for input saved from start of block')

            # get the current module
            # TODO: is this very bad for latency? Only load module if you have to 
            curr_layer = get_current_module(model, imodule)

            # update communication I/O for this layer  
            # TODO: revist this implementation
            split_param_name = layer_names_fx[imodule] + '.weight'
            if split_param_name in split_module_names:
                # skip if machine doesnt expect input
                if len(configs['partition'][split_param_name]['channel_id'][imach]) == 0:
                        print(f'\t\t-WARNING: No input assigned to this machine (but it was sent input?). Skipping...')
                        continue

                # TODO: reconsider implementation 
                # What input channels does this machine compute?
                input_channels = torch.tensor(configs['partition'][split_param_name]['channel_id'][imach],
                        device=torch.device(configs['device']))
                N_in = len(input_channels) # TODO: is this used?

                # Where to send output (map of output channels to different machines)
                output_channel_map = configs['partition'][split_param_name]['filter_id']
            elif type(curr_layer) == nn.Linear and imodule == total_layers_fx-1:
                # if final layer output all goes to machine 0 
                # TODO: find better way to handle this. Also will we encounter Linear layers not at the end of the model
                N_Cin = curr_layer.in_features
                Cin_per_machine = N_Cin/num_machines
                if Cin_per_machine % 1 > 0:
                        print('ERROR: UNEXPECTED NUMBER OF I/O FOR LINEAR MODULE {imodule}')
                Cin_per_machine = int(Cin_per_machine)
                input_channels = np.arange(Cin_per_machine) + imach*Cin_per_machine
                N_Cout = curr_layer.out_features 
                output_channel_map = [None]*num_machines
                for i in range(num_machines):
                        if i == 0:
                            # send all outputs to machine 0 for final layer
                            # TODO: revisit this implementation choice
                            output_channel_map[i] = np.arange(N_Cout) 
                        else:
                            output_channel_map[i] = np.array([])
                input_channels = torch.tensor(input_channels, device=torch.device(configs['device']))
            else:
                # for batch normal, and functional passes through the code
                # TODO: address the following assumptions:
                #       - assume all BN layers have C_in divisable by num_machines
                #       - assume C_in are evenly split in sequential order WARNING THIS WILL BREAK WHEN WE START TO DO ASSIGN WEIGHTS TO DIFF MACHINES
                N_Cin = curr_layer.num_features
                Cin_per_machine = N_Cin/num_machines
                if Cin_per_machine % 1 > 0:
                        print('ERROR: UNEXPECTED NUMBER OF I/O FOR BATCH NORMAL MODULE {imodule}')
                Cin_per_machine = int(Cin_per_machine)
                input_channels = np.arange(Cin_per_machine) + imach*Cin_per_machine
                output_channel_map = [None]*num_machines
                for i in range(num_machines):
                        if i == imach:
                                output_channel_map[i] = input_channels
                        else:
                                output_channel_map[i] = np.array([])
                input_channels = torch.tensor(input_channels, device=torch.device(configs['device']))


            # make vertically split layer 
            if type(curr_layer) == nn.Conv2d:
                split_layer = split_conv_layer(curr_layer, input_channels)
            elif type(curr_layer) == nn.BatchNorm2d:
                split_layer = split_bn_layer(curr_layer, input_channels)
            elif type(curr_layer) == nn.Linear:
                split_layer = split_linear_layer(curr_layer, input_channels)
            else:
                print(f'\t\t-Skipping module {type(curr_layer).__name__}')
                continue
                        
            # eval split
            split_layer.eval()
            out_tensor = split_layer(curr_input.index_select(1, input_channels))
            if type(curr_layer) == nn.BatchNorm2d:
                tmp_out_tensor = torch.zeros(curr_input.shape, dtype=torch.float64)
                tmp_out_tensor[:,input_channels.numpy(),:,:] = out_tensor
                out_tensor = tmp_out_tensor

            print(f'\t\t Output tensor shape : {out_tensor.shape}')

            # debug
            nonzero_out_tensor = torch.unique(torch.nonzero(out_tensor, as_tuple=True)[1])

            # look at which C_out need to be computed and sent
            #nonzero_Cout = torch.unique(torch.nonzero(split_layer.weight, as_tuple=True)[0]) # find nonzero dimensions in output channels
            nonzero_Cout = get_nonzero_channels(out_tensor)

            # prep communications by populating output
            out_channel_array = torch.arange(out_tensor.shape[1])
            for rx_mach in range(num_machines):
                # only add to output if communication is necessary 

                # Get output channels for current rx machine? TODO: consider removing, this just maps C_out's to machine
                output_channels = torch.tensor(output_channel_map[rx_mach],
                        device=torch.device(configs['device']))

                # TODO: is there a faster way to do this? Consider putting larger array 1st... just not sure which one that'd be
                nonzero_out_channels = nonzero_Cout[torch.isin(nonzero_Cout, output_channels)]
                if nonzero_out_channels.nelement() > 0:
                        communication_mask = torch.isin(out_channel_array, nonzero_out_channels)

                        # TODO: this is inefficient, redo. Probbably need to send a tensor and some info what output channels are being sent
                        tmp_out = torch.zeros(out_tensor.shape, dtype= torch.float64) 
                        if imodule == total_layers_fx-1:
                                tmp_out[:,communication_mask] = out_tensor[:,communication_mask]
                        else:
                                tmp_out[:,communication_mask,:,:] = out_tensor[:,communication_mask,:,:]
                        output[rx_mach][imach] = tmp_out

                        # debug
                        print(f'\t\t sending C_out {nonzero_out_channels} to machine {rx_mach}')

        # send to next layer  
        input = output
        print(f'Finished execution of layer {imodule}')

        # update timing
        layer_completion_time_stamp[layer_names_fx[imodule]] = time.time()
        if imodule > 0:
            layer_execution_duration[layer_names_fx[imodule]] = layer_completion_time_stamp[layer_names_fx[imodule]] - layer_completion_time_stamp[layer_names_fx[imodule-1]] 
        else:
            layer_execution_duration[layer_names_fx[imodule]] = layer_completion_time_stamp[layer_names_fx[imodule]] - split_execution_start_time

        # check output at end of each layer to see if fit matches 
        tmp_output = input 
        need_to_init  = True
        for rx_mach in range(num_machines):
                for tx_mach in range(num_machines):
                        if not tmp_output[rx_mach][tx_mach] == None:
                                if need_to_init:
                                        vert_output = tmp_output[rx_mach][tx_mach]
                                        need_to_init = False
                                else:
                                        # TODO: += causes assignment issues, switched to x = x+y which might be more more inefficent memory wise ... 
                                        vert_output = vert_output + tmp_output[rx_mach][tx_mach] 
                                        #nz_channels = get_nonzero_channels(vert_output)
                                        #print(f'({rx_mach},{tx_mach}) {nz_channels}')
        
        if imodule == total_layers_fx-1:
            # apply bias
            # TODO: assumes Linear layer is final layer and bias can be handled as final step 
            vert_output = vert_output + get_current_module(model, imodule).bias
            
            # final execution time
            tot_split_execution_time = time.time() - split_execution_start_time
            print(f'\n\n############################# FINAL EXECUTION TIME {tot_split_execution_time} [seconds] #############################\n\n')

        truth_output = horz_output[layer_names_fx[imodule]]
        if 'x' == layer_names_fx[imodule]:
            print(f'Input layer. Skipping comparison')
        elif torch.is_tensor(truth_output):
            max_diff, max_by_Cout = compare_outputs(vert_output, truth_output)
            if max_diff > 0.1:
                BREAK_LOOP = 1 
        else:
            print(f'Horizontal output is {type(truth_output)}. Skipping comparison')
        print()

        if BREAK_LOOP:
            break


Executing module 0: x
	Executing on machine 0
		 received input channels tensor([0, 1, 2])
		-model input layer.. skipping
	Executing on machine 1
		 received input channels tensor([0, 1, 2])
		-model input layer.. skipping
	Executing on machine 2
		 received input channels tensor([0, 1, 2])
		-model input layer.. skipping
	Executing on machine 3
		 received input channels tensor([0, 1, 2])
		-model input layer.. skipping
Finished execution of layer 0
Input layer. Skipping comparison

Executing module 1: conv1
	Executing on machine 0
		 received input channels tensor([0, 1, 2])
		-WARNING: No input assigned to this machine (but it was sent input?). Skipping...
	Executing on machine 1
		 received input channels tensor([0, 1, 2])
		 Output tensor shape : torch.Size([1, 64, 32, 32])
		 sending C_out tensor([16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31]) to machine 1
		 sending C_out tensor([56, 62]) to machine 3
	Executing on machine 2
		 received input channels tensor([0, 1

In [40]:
'''
    Conv1 layer test -- should we expect some difference, or exactly 0 in the output when we split the model?
'''

# DIFFERENCE SHOULD BE 0 NOT 1E-7

N_in = 1
split_1 = nn.Conv2d(N_in,
            model.conv1.weight.shape[0], # TODO does this need to be an int? (currently tensor)
            kernel_size= model.conv1.kernel_size,
            stride=model.conv1.stride,
            padding=model.conv1.padding, 
            bias=False) # TODO: add bias during input collecting step on next layer 
split_1.weight = torch.nn.Parameter(model.conv1.weight.index_select(1, torch.tensor([0])))  
out_split1 = split_1(input_tensor.index_select(1, torch.tensor([0])))

split_2 = split_1
split_2.weight = torch.nn.Parameter(model.conv1.weight.index_select(1, torch.tensor([1])))  
out_split2 = split_2(input_tensor.index_select(1, torch.tensor([1])))

split_3 = split_1
split_3.weight = torch.nn.Parameter(model.conv1.weight.index_select(1, torch.tensor([2])))  
out_split3 = split_3(input_tensor.index_select(1, torch.tensor([2])))

split_out = torch.add(torch.add(out_split1, out_split2), out_split3)
full_out = model.conv1(input_tensor)

diff_output = torch.abs(full_out - split_out)
max_diff = torch.max(diff_output)
max_diff.sci_mode = True
print(max_diff)

tensor(4.4409e-16, dtype=torch.float64, grad_fn=<MaxBackward1>)


In [41]:
'''  
    Inspect I/O of single layer
'''

t = torch.ones((1,2,2,2), dtype=torch.float32) # (batch, in channel, H, W)
w = torch.ones((1,2,2,2), dtype=torch.float32) # (out channels, in channels, H, W)
w[0,0,0,0] = 1e-10
w[0,0,0,1] = 1e-10
w[0,1,0,0] = 1e-10

full_conv = torch.nn.Conv2d(2,1,kernel_size=(2,2), bias=False, stride=(1),dtype=torch.float32)
full_conv.weight = torch.nn.Parameter(w)
conv1 = torch.nn.Conv2d(1,1,kernel_size=(2,2), bias=False, stride=1, dtype=torch.float32)
conv1.weight =torch.nn.Parameter( w[:,0:1,:,:])
conv2 = torch.nn.Conv2d(1,1,kernel_size=(2,2), bias=False, stride=1,dtype=torch.float32)
conv2.weight = torch.nn.Parameter(w[:,1:2,:,:])

full_conv.eval()
conv1.eval()
conv2.eval()

with torch.no_grad():
    full_out = full_conv(t)
    split_out =  conv2(t[0,1:2,:,:]) + conv1(t[:,0:1,:,:])

diff = torch.abs(full_out - split_out)

torch.nonzero(diff)
#print()


tensor([], size=(0, 4), dtype=torch.int64)